In [25]:
from dotenv import load_dotenv
load_dotenv()
from langchain_google_genai import ChatGoogleGenerativeAI    
from pathlib import Path  

In [26]:
import json
import re

In [27]:
current_dir =Path.cwd() 
persist_directory = current_dir /'db'/ 'main_db'
persist_directory=str(persist_directory)

In [28]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings
embeddings = VertexAIEmbeddings(model="text-embedding-004")

In [30]:
from langchain_chroma import Chroma
vector_store= Chroma(persist_directory=persist_directory,
            embedding_function=embeddings)

In [31]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.1}, 
)

In [32]:
from langchain_core.tools import tool

In [33]:
# langraph application
from langchain_core.prompts import ChatPromptTemplate
# from langchain_core.pydantic_v1 import Field
from pydantic import BaseModel, Field
from typing import Literal

In [34]:

from typing import List
from typing_extensions import TypedDict

In [35]:
class GraphState(TypedDict):
    """
    Rrepresent the state of the graph.
    
    Attributes:
    
    
    """
    input :str
    all_documents: List[str]
    query_list:List[str]
    output:str
    extraction:List[dict]    
    
    

In [36]:
def  generating_qureys(state:GraphState):
    """
    Generate qureys on a text to get more information
    """
    print("---generating_qureys---")
    large_query= state["input"]
    
    query_list = llm.invoke(f"You task is to return a json object in format of [query1,query2...],dont give any other  than json string not also any type of text other than the json string,query1 ,query2 are the querys which are generated such that will use to get more information from the RAG.example:send mailto.Database contain the information on people context any many techical documents text:{large_query}")
    raw_output = query_list.content

# Optional: remove prefix like "json" or anything before the actual JSON array
    match = re.search(r'\[.*\]', raw_output, re.DOTALL)
    queries = []
    if match:
     cleaned_json_str = match.group(0)
     queries = json.loads(cleaned_json_str)
     print(queries)
    else:
     print("No valid JSON array found in the output.")

    print("queries",queries)
    state['query_list']=queries
    
    return state
    

In [37]:
def multi_query_retrival(state:GraphState):
    """
    Retrive relavent document from the RAG from multiple querys
    """
    print("---multi_query_retrival---")
    query_list=state["query_list"]
    all_documents=[]
    
    for x in query_list:
        documents = retriever.invoke(x)
        all_documents.append(documents)
    print("all_documnets",all_documents)
    state["all_documents"] = all_documents
    return state

In [38]:
def generator(state:GraphState):
    '''
    generate a final output combininig retrive documents and the retrive documnets
    '''
    print("---generator---")
    input= state["input"]
    all_documents = state["all_documents"]
    
    
    result = llm.invoke(f"your task is to add details into the original text,Do not mention were you got the information formDo not mention were you got the information form.only provide details about the things menstioned,text{input},context:{all_documents}")
    output = result.content
    print("output",output)
    state["output"]=output
    
    return state
    

In [39]:
def extractor(state:GraphState):
    """
    Extract the information from the text in json format
    """
    print("---extractor---")
    output= state["output"]
    print("output",output)
    promt = 'You are an email information extractor, will be given a text, you have to extract the information in format: [{"email":"examplename@exaple.com","body":"body of mail","subject":"subject of the mail"},{"email":"examplename@exaple.com","body":"body of mail","subject":"subject of the mail"}],do not give nay other thing other than the json string,if not found any email in the text, return json[{"nothing found":"no email found"}],text'
    result = llm.invoke(f"{promt}, text{output}")
    output = result.content
    raw_output = output 
    print("raw_output",raw_output)

# Optional: remove prefix like "json" or anything before the actual JSON array
    match = re.search(r'\[.*\]', raw_output, re.DOTALL)
    queries = []
    if match:
     cleaned_json_str = match.group(0)
     queries = json.loads(cleaned_json_str)
     print("--->",queries)
    else:
     print("No valid JSON array found in the output.")

    print("extraction",queries)
    state['extraction']=queries
    
    return state

In [40]:
# creating flow

from langgraph.graph import END, StateGraph, START

In [41]:
workflow = StateGraph(GraphState)

In [42]:
workflow.add_node("generating_qureys",generating_qureys)
workflow.add_node("multi_query_retrival",multi_query_retrival)
workflow.add_node("generator",generator)
workflow.add_node("extractor",extractor)

In [43]:
workflow.add_edge(START,"generating_qureys")
workflow.add_edge("generating_qureys","multi_query_retrival")
workflow.add_edge("multi_query_retrival","generator")
workflow.add_edge("generator","extractor")
workflow.add_edge("extractor",END)

In [44]:
# Compile
app = workflow.compile()

In [45]:
from pprint import pprint

In [46]:

# Run
all_states=[]

stkd='''
hello guys,
thankyou for jointing the meeting.
i just what to tell to send mail to partha kadam whose email id is parthakadam2007@gmail.com wishing him happy birthday.
'''

inputs = {"input": stkd}
final_message = None  # Keep track of final generation manually

for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Node '{key}':")
        
        # If this is the generation node and has messages
        if isinstance(value, dict) and "messages" in value:
            final_message = value["messages"][-1].content
    all_states.append(value)
    pprint("\n---\n")
    

# Print final message if available
if final_message:
    pprint("Assistant: " + final_message)
else:
    pprint("No final assistant message found.")

---generating_qureys---
["what is partha kadam's email address?", 'find the email address for partha kadam']
queries ["what is partha kadam's email address?", 'find the email address for partha kadam']
"Node 'generating_qureys':"
'\n---\n'
---multi_query_retrival---
all_documnets [[Document(id='1', metadata={'source': 'c:\\Users\\DELL\\Desktop\\CloneFlow\\production\\user_settings\\user_info.txt'}, page_content='Partha Kadam, a seventeen-year-old living at 123 Main Street in Springfield, USA, can be reached at partha.kadam@example.com or 123-456-7890.'), Document(id='aea8993e-4d96-4224-b027-d954f301b0fc', metadata={'source': 'c:\\Users\\DELL\\Desktop\\CloneFlow\\production\\uploads\\dataBasePassword.txt'}, page_content='AI Email with Partha      J9wIP7bZWTCSg0L4'), Document(id='d88748d6-fc4c-4ff9-8fed-543653f13fdb', metadata={'source': 'c:\\Users\\DELL\\Desktop\\CloneFlow\\production\\uploads\\dataBasePassword.txt'}, page_content='AI Email with Partha      J9wIP7bZWTCSg0L4')], [Documen

In [47]:
all_states[-1]['extraction'] 

[{'email': 'parthakadam2007@gmail.com',
  'body': 'Please make sure your email wishes him a happy birthday.',
  'subject': 'Birthday email for Partha Kadam'},
 {'email': 'partha.kadam@example.com',
  'body': 'Please make sure your email wishes him a happy birthday.',
  'subject': 'Birthday email for Partha Kadam'}]

In [48]:
all_states[2]["output"]

'Hello guys,\n\nThank you for joining the meeting.\n\nI just wanted to request that you send a birthday email to Partha Kadam.  While his email address is listed as parthakadam2007@gmail.com,  another email address on file is partha.kadam@example.com.  Please use whichever address you can reach him at.  Partha is seventeen years old and lives at 123 Main Street in Springfield, USA.  His phone number is 123-456-7890, though this is likely less reliable for contacting him than email.  Please make sure your email wishes him a happy birthday.'